In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_validate 
import random 
from sklearn.model_selection import KFold  
from sklearn.ensemble import RandomForestClassifier

load dataset from seaborn

In [10]:
df = sns.load_dataset('penguins')
df.head(3)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female


In [11]:
df.dropna(inplace = True) 
df.duplicated().sum()

0

In [15]:
X = df[['bill_length_mm' ,	'bill_depth_mm'	, 'flipper_length_mm']]  
y = df['species']  
max_feat = range(1,100,1)  
max_feat = [x/100 for x in max_feat] 
init = 0 
tmax = 300 
ctemp = 0 
cv = KFold(n_splits=5, shuffle=True) 
init=0   
max_depths = [x for x in range(1,50)] 
n_estimator = [x for x in range(1,100)] 
init_max_feat = 0
init_max_depth = 0 
init_n_estimators = 0 
max = 0
model = RandomForestClassifier(max_features=max_feat[init_max_feat],
                                   random_state=1, 
                                   n_estimators= n_estimator[init_n_estimators],      #model initialization
                                   max_depth = max_depths[init_max_depth])  
while(tmax > 0)  :   
    next_init = {'next_init_max_depth' : np.random.randint(0,len(max_depths)), 
                 'next_init_n_estimators' : np.random.randint(0, len(n_estimator)),  #random every params for next iterations
                 'next_init_max_feat' : np.random.randint(0, len(max_feat))}  
   
    next_model = RandomForestClassifier(max_features=max_feat[next_init['next_init_max_feat']],
                                         random_state=1, 
                                         n_estimators = n_estimator[next_init['next_init_n_estimators']], #initiate next model with randomly generated value.
                                         max_depth = max_depths[next_init['next_init_max_depth']]) 
    c_eval = cross_validate(estimator = model , X = X,y=y, scoring='accuracy', return_train_score=True, cv=cv) 
    n_eval = cross_validate(estimator = next_model , X = X,y=y, scoring='accuracy', return_train_score=True, cv=cv) 
    current_score = np.mean(c_eval['test_score']) 
    next_score = np.mean(n_eval['test_score']) 

    if (tmax ==300) : 
      max = current_score  #max initialization 
      best_model = model
      temperature = tmax

    else : 
      if (current_score > max) : 
         max = current_score     #climb. 
         best_temp = tmax 
         best_model = model
         temperature = tmax

    score_diff = next_score - current_score 
    if (score_diff > 0) : 
       model = next_model  

    if (tmax%50==0) : 
       print('current ') 
       print('max_depth : ', model.get_params()['max_depth']) 
       print('n_estimator : ', model.get_params()['n_estimators'])
       print('max_feat : ', model.get_params()['max_features'])
       print('temperature : ', tmax) 
       print('current_score : ', current_score) 
       
       print('='*90) 

       print('best ') 
       print('max_depth : ', best_model.get_params()['max_depth']) 
       print('n_estimator : ', best_model.get_params()['n_estimators'])
       print('max_feat : ', best_model.get_params()['max_features'])
       print('temperature : ', temperature) 
       print('max_score : ', max) 
       print('='*90)
    tmax = tmax - 1

    
    

current 
max_depth :  43
n_estimator :  68
max_feat :  0.89
temperature :  300
current_score :  0.7327001356852103
best 
max_depth :  1
n_estimator :  1
max_feat :  0.01
temperature :  300
max_score :  0.7327001356852103
current 
max_depth :  8
n_estimator :  93
max_feat :  0.58
temperature :  250
current_score :  0.9699231117141565
best 
max_depth :  31
n_estimator :  23
max_feat :  0.05
temperature :  298
max_score :  0.9849841700587969
current 
max_depth :  40
n_estimator :  90
max_feat :  0.05
temperature :  200
current_score :  0.9729534147444596
best 
max_depth :  31
n_estimator :  23
max_feat :  0.05
temperature :  298
max_score :  0.9849841700587969
current 
max_depth :  12
n_estimator :  95
max_feat :  0.42
temperature :  150
current_score :  0.9789687924016282
best 
max_depth :  31
n_estimator :  23
max_feat :  0.05
temperature :  298
max_score :  0.9849841700587969
current 
max_depth :  12
n_estimator :  76
max_feat :  0.32
temperature :  100
current_score :  0.9789687924016

In [16]:
best_model

RandomForestClassifier(max_depth=31, max_features=0.05, n_estimators=23,
                       random_state=1)